In [ ]:
import nltk
import difflib
import time
import gc
import itertools
import multiprocessing
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, StratifiedKFold

from models_utils_fe import *
from models_utils_skf import *

In [ ]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'
feats_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/data/features/uncleaned/'
trans_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/data/features/lemmatized_fullclean/transformations/'

X_train = pd.read_pickle('Xtrain_814colsBest.pkl', compression = 'bz2')
xgb_feats = pd.read_csv(feats_src + '/the_1owl/owl_train.csv')
y_train = xgb_feats[['is_duplicate']]

train_interactions = pd.read_pickle(src + 'train_tony_interaction_gru.pkl')

X_train = pd.concat([X_train, train_interactions], axis = 1)
X_train = X_train.astype('float32')

del xgb_feats
gc.collect()

XGB: Final score for fold 1 : 0.179168044048 

In [ ]:
lgb_params = {
    'task' : 'train',
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : {'binary_logloss'},
    'learning_rate' : 0.01,
    'feature_fraction' : 0.51,
    'bagging_fraction': 0.9,
    'bagging_freq': 100,
    'num_leaves' : 255,
    'max_depth': 8,
    'min_data_in_leaf': 23,
    'subsample': 0.8,
    'colsample_bytree': 0.41,
    'silent': 1,
    'random_state': 1337,
    'verbose': 1,
    'nthread': 9,
}

xgb_params = {
    'seed': 1337,
    'colsample_bytree': 0.42,
    'silent': 1,
    'subsample': 0.85,
    'eta': 0.02,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 8,
    'min_child_weight': 20,
    'nthread': 4,
    'tree_method': 'hist',
    }

In [ ]:
xgb_foldrun(X_train, y_train, xgb_params, 'FredFeatsGRU')

In [ ]:
lgb_foldrun(X_train, y_train, lgb_params, 'FredFeatsGRU')